In [1]:
import argparse
import pandas as pd
import numpy as np
from typing import List

In [2]:
np.random.seed(0)

test_entity_ids_file = 'entity_ids.txt'
test_read_count_paths_file = 'read_count_paths.txt'
test_clustering_table_file = 'clustering_table.tsv'
test_training_blacklist_file = 'training_blacklist.txt'

test_num_samples = 10000
test_entity_ids = np.array(['sample_{:d}'.format(i) 
                            for i in range(1, test_num_samples + 1)])
np.savetxt(test_entity_ids_file, test_entity_ids, fmt='%s')

test_read_count_paths = ['gs://fake-bucket/sample_{:d}.counts.tsv'.format(i) 
                         for i in range(1, test_num_samples + 1)]
np.savetxt(test_read_count_paths_file, test_read_count_paths, fmt='%s')

test_num_clusters = 30
test_dirichlet_alpha = 0.001
test_around_num_digits = 3
test_cluster_names = ['CLUSTER_{:d}'.format(i) 
                      for i in range(1, test_num_clusters + 1)]
test_clustering_df = pd.DataFrame(columns=['SAMPLE_NAME'] + test_cluster_names)
test_unnorm_responsibilities = np.around(np.random.dirichlet(test_dirichlet_alpha * np.ones(test_num_clusters), 
                                                             size=test_num_samples), 
                                         test_around_num_digits)
test_responsibilities = test_unnorm_responsibilities / np.sum(test_unnorm_responsibilities, axis=1)[:, np.newaxis]
test_clustering_df['SAMPLE_NAME'] = test_entity_ids
test_clustering_df[test_cluster_names] = test_responsibilities
test_clustering_df.to_csv(test_clustering_table_file, sep='\t', index=False)

test_training_blacklist_fraction = 0.02
test_training_blacklist = test_entity_ids[1 == np.random.choice([0, 1], 
                                                                p=[1 - test_training_blacklist_fraction, test_training_blacklist_fraction], 
                                                                size=test_num_samples)]
np.savetxt(test_training_blacklist_file, test_training_blacklist, fmt='%s')

In [3]:
def determine_cohorts_and_cases_and_write_results(output_dir: str,
                                                  entity_ids_file: List[str],
                                                  read_count_paths_file: List[str],
                                                  clustering_table_file: str, 
                                                  training_blacklist_file: str,
                                                  number_of_training_samples_per_model: int):
    #load all files
    entity_ids = np.loadtxt(entity_ids_file, dtype=str)
    read_count_paths = np.loadtxt(read_count_paths_file, dtype=str)
    assert len(entity_ids) == len(read_count_paths), \
        "Number of entity IDs and number of read count paths must be equal."
    clustering_table_df = pd.read_csv(clustering_table_file, sep='\t')
    cluster_names = clustering_table_df.columns[1:]
    training_blacklist = np.loadtxt(training_blacklist_file, dtype=str)
    
    results_df = pd.DataFrame(columns=['SAMPLE_NAME', 
                                       'READ_COUNT_PATH', 
                                       'CLUSTER_MEMBERSHIP', 
                                       'IN_TRAINING_BLACKLIST', 
                                       'IN_RESCUE_LIST', 
                                       'TRAINING_SAMPLE'])
    
    results_df['SAMPLE_NAME'] = entity_ids
    results_df['READ_COUNT_PATH'] = read_count_paths
    results_df['CLUSTER_MEMBERSHIP'] = clustering_table_df[cluster_names].idxmax(axis=1) # use MAP responsibility
    results_df['IN_TRAINING_BLACKLIST'] = results_df['SAMPLE_NAME'].isin(training_blacklist)
    number_of_blacklisted_samples_per_cluster = results_df.groupby('CLUSTER_MEMBERSHIP')['IN_TRAINING_BLACKLIST'].sum()
    number_of_available_samples_per_cluster = results_df['CLUSTER_MEMBERSHIP'].value_counts() - number_of_blacklisted_samples_per_cluster
    rescue_clusters = number_of_available_samples_per_cluster[number_of_available_samples_per_cluster < number_of_training_samples_per_model].keys()
    results_df['IN_RESCUE_LIST'] = results_df['CLUSTER_MEMBERSHIP'].isin(rescue_clusters)
    
    training_samples = results_df[~results_df['IN_TRAINING_BLACKLIST'] & ~results_df['IN_RESCUE_LIST']] \
                                 .groupby('CLUSTER_MEMBERSHIP') \
                                 .apply(lambda x: x.sample(number_of_training_samples_per_model))['SAMPLE_NAME']
            
    results_df['TRAINING_SAMPLE'] = results_df['SAMPLE_NAME'].isin(training_samples)
    
    assert not any(results_df['IN_TRAINING_BLACKLIST'] & results_df['TRAINING_SAMPLE'])
    assert not any(results_df['IN_RESCUE_LIST'] & results_df['TRAINING_SAMPLE'])
    
    print(results_df)
    #output complete table
    
    #output entity id and read count path files for both training and case samples for each cluster 

In [4]:
output_dir = '.'
number_of_training_samples_per_model = 300

determine_cohorts_and_cases_and_write_results(output_dir,
                                              test_entity_ids_file,
                                              test_read_count_paths_file,
                                              test_clustering_table_file, 
                                              test_training_blacklist_file,
                                              number_of_training_samples_per_model)

       SAMPLE_NAME                           READ_COUNT_PATH  \
0         sample_1      gs://fake-bucket/sample_1.counts.tsv   
1         sample_2      gs://fake-bucket/sample_2.counts.tsv   
2         sample_3      gs://fake-bucket/sample_3.counts.tsv   
3         sample_4      gs://fake-bucket/sample_4.counts.tsv   
4         sample_5      gs://fake-bucket/sample_5.counts.tsv   
5         sample_6      gs://fake-bucket/sample_6.counts.tsv   
6         sample_7      gs://fake-bucket/sample_7.counts.tsv   
7         sample_8      gs://fake-bucket/sample_8.counts.tsv   
8         sample_9      gs://fake-bucket/sample_9.counts.tsv   
9        sample_10     gs://fake-bucket/sample_10.counts.tsv   
10       sample_11     gs://fake-bucket/sample_11.counts.tsv   
11       sample_12     gs://fake-bucket/sample_12.counts.tsv   
12       sample_13     gs://fake-bucket/sample_13.counts.tsv   
13       sample_14     gs://fake-bucket/sample_14.counts.tsv   
14       sample_15     gs://fake-bucket/

In [4]:
def main():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--output_dir', 
                        type=str,    
                        help='Output directory')
    
    parser.add_argument('--entity_ids_file', 
                        type=str,
                        help='File containing entity IDs corresponding to read count paths')
    
    parser.add_argument('--read_count_paths_file', 
                        type=str,
                        help='File containing read count paths (output of GATK CollectReadCounts) corresponding to entity IDs')

    parser.add_argument('--clustering_table_file',
                        type=str,
                        help='Table of clustered samples (output of ClusterSamplesFromCoverage workflow)')
    
    parser.add_argument('--training_blacklist_file',
                        type=str,
                        help='File containing blacklist of entity IDs for samples not to be used for training models')
    
    parser.add_argument('--number_of_training_samples_per_model',
                        type=int,
                        help='Number of samples used to train each model; samples in clusters with a number of ' +
                             'non-blacklisted samples less than this will be output to a rescue list')

    args = parser.parse_args()
 
    output_dir = args.output_dir
    entity_ids_file = args.entity_ids_file
    read_count_paths_file = args.read_count_paths_file
    clustering_table_file = args.clustering_table_file
    training_blacklist_file = args.training_blacklist_file
    number_of_training_samples_per_model = args.number_of_training_samples_per_model
    
    assert number_of_training_samples_per_model > 0, "Number of training samples per model must be positive."

    determine_cohorts_and_cases_and_write_results(output_dir,
                                                  entity_ids_file,
                                                  read_count_paths_file,
                                                  clustering_table_file, 
                                                  training_blacklist_file,
                                                  number_of_training_samples_per_model)

if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [--output_dir OUTPUT_DIR]
                             [--entity_ids ENTITY_IDS [ENTITY_IDS ...]]
                             [--read_count_files READ_COUNT_FILES [READ_COUNT_FILES ...]]
                             [--clustering_table_file CLUSTERING_TABLE_FILE]
                             [--training_blacklist_file TRAINING_BLACKLIST_FILE]
                             [--number_of_training_samples_per_model NUMBER_OF_TRAINING_SAMPLES_PER_MODEL]
                             [--minimum_number_of_samples_per_cluster MINIMUM_NUMBER_OF_SAMPLES_PER_CLUSTER]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1890032050/jupyter/kernel-8383480e-4347-4106-809e-38fa9d8a48ad.json


SystemExit: 2

/home/BROAD.MIT.EDU/slee/.pyenv/versions/anaconda3-4.3.1/envs/gatk-ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
